# 🔬 DeepBridge Resilience Testing Demo

<div style="background-color: #e8f5e9; padding: 15px; border-radius: 5px; border-left: 5px solid #4caf50;">
<b>📓 Notebook Purpose</b><br>
<b>Goal:</b> Demonstrate how to run resilience tests with DeepBridge<br>
<b>Level:</b> Beginner<br>
<b>Time:</b> 5-10 minutes<br>
<b>Dataset:</b> Breast Cancer (sklearn)
</div>

---

## What is Resilience Testing?

**Resilience testing** evaluates how robust your model is to:
- 📊 **Data drift** - Changes in feature distributions
- 🔄 **Concept drift** - Changes in relationships between features and target
- 📉 **Performance degradation** - Model stability over time
- 🎯 **Distribution shifts** - Different data characteristics in production

### Why is this important?

In production, your model will encounter:
- Data from different time periods
- Data with different statistical properties
- Edge cases and outliers

**DeepBridge's resilience test** helps you understand if your model will remain reliable when faced with these challenges!

---

## 1. 🛠️ Setup & Imports

In [1]:
# Standard imports
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

# DeepBridge
from deepbridge import DBDataset, Experiment

# Configuration
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ Setup complete!")
print("📦 DeepBridge imported successfully")

2025-11-12 16:21:33,052 - deepbridge.reports - DEBUG - Using refactored FairnessDataTransformer
2025-11-12 16:21:33,554 - deepbridge.reports - INFO - Successfully imported radar chart fix
2025-11-12 16:21:33,556 - deepbridge.reports - INFO - Successfully patched EnhancedUncertaintyCharts.generate_model_metrics_comparison
2025-11-12 16:21:33,557 - deepbridge.reports - INFO - Successfully applied enhanced_charts patch
2025-11-12 16:21:33,561 - deepbridge.reports - INFO - Successfully loaded UncertaintyChartGenerator
2025-11-12 16:21:33,563 - deepbridge.reports - INFO - Successfully imported and initialized SeabornChartGenerator
2025-11-12 16:21:33,563 - deepbridge.reports - INFO - SeabornChartGenerator has_visualization_libs: True
2025-11-12 16:21:33,564 - deepbridge.reports - INFO - Available chart methods: ['bar_chart', 'boxplot_chart', 'coverage_analysis_chart', 'detailed_boxplot_chart', 'distribution_grid_chart', 'feature_comparison_chart', 'feature_importance_chart', 'feature_psi_ch

## 2. 📊 Load Dataset & Train Model

In [2]:
# Load Breast Cancer dataset
print("📊 Loading Breast Cancer dataset...\n")
cancer = load_breast_cancer()
df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
df['target'] = cancer.target

print(f"Dataset shape: {df.shape}")
print(f"Features: {len(cancer.feature_names)}")
print(f"Classes: {list(cancer.target_names)}")
print(f"Class distribution:\n{df['target'].value_counts()}")

# Display first few rows
print("\nFirst rows:")
df.head()

📊 Loading Breast Cancer dataset...

Dataset shape: (569, 31)
Features: 30
Classes: [np.str_('malignant'), np.str_('benign')]
Class distribution:
target
1    357
0    212
Name: count, dtype: int64

First rows:


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [3]:
# Train a Random Forest model
print("🌲 Training Random Forest model...\n")

model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

# Separate features and target
X = df.drop('target', axis=1)
y = df['target']

# Train on full dataset (DeepBridge will split internally)
model.fit(X, y)

# Quick evaluation
y_pred = model.predict(X)
y_proba = model.predict_proba(X)[:, 1]

train_acc = accuracy_score(y, y_pred)
train_auc = roc_auc_score(y, y_proba)

print(f"✅ Model trained successfully!")
print(f"   Training Accuracy: {train_acc:.3f}")
print(f"   Training ROC AUC: {train_auc:.3f}")
print(f"\n💡 Now let's test its resilience!")

🌲 Training Random Forest model...

✅ Model trained successfully!
   Training Accuracy: 1.000
   Training ROC AUC: 1.000

💡 Now let's test its resilience!


## 3. 🔬 Run DeepBridge Resilience Test

DeepBridge will automatically:
- ✅ Split your data into train/test sets
- ✅ Analyze feature distributions
- ✅ Test model performance on perturbed data
- ✅ Detect potential drift issues
- ✅ Calculate resilience scores
- ✅ Generate comprehensive reports

In [4]:
# Step 1: Create DBDataset
print("📦 Creating DBDataset...\n")

dataset = DBDataset(
    data=df,
    target_column='target',
    model=model,
    test_size=0.2,
    random_state=RANDOM_STATE
)

print("✅ DBDataset created successfully!")
print(f"   Total samples: {len(df)}")
print(f"   Features: {len(cancer.feature_names)}")
print(f"   Model: {type(model).__name__}")

📦 Creating DBDataset...

✅ DBDataset created successfully!
   Total samples: 569
   Features: 30
   Model: RandomForestClassifier


In [5]:
# Step 2: Create Experiment
print("🧪 Creating Experiment...\n")

exp = Experiment(
    dataset=dataset,
    experiment_type='binary_classification',
    test_size=0.2,
    random_state=RANDOM_STATE,
    tests=['resilience']  # Specify which tests to prepare
)

print("✅ Experiment initialized!")
print("   Ready to run resilience tests...")

🧪 Creating Experiment...

✅ Initial model evaluation complete: RandomForestClassifier
✅ Experiment initialized!
   Ready to run resilience tests...


In [6]:
# Step 3: Run Resilience Test
print("🔬 Running Resilience Test...\n")
print("This will test your model against:")
print("  • Data distribution shifts")
print("  • Feature perturbations")
print("  • Performance stability")
print("\nPlease wait...\n")

# Run tests with 'quick' configuration
# Since we specified tests=['resilience'], only resilience test will run
results = exp.run_tests(config_name='quick')

print("\n" + "="*70)
print("✅ RESILIENCE TEST COMPLETED!")
print("="*70)

# Access resilience results
if hasattr(results, 'results') and 'resilience' in results.results:
    resilience_data = results.results['resilience']
    
    print("\n📊 Key Findings:\n")
    
    # Check if resilience_data has results attribute
    if hasattr(resilience_data, 'results'):
        res = resilience_data.results
    else:
        res = resilience_data if isinstance(resilience_data, dict) else {}
    
    # Model performance
    if 'model_performance' in res:
        perf = res['model_performance']
        print(f"   Model Performance:")
        if 'accuracy' in perf:
            print(f"      • Accuracy: {perf['accuracy']:.3f}")
        if 'roc_auc' in perf:
            print(f"      • ROC AUC: {perf['roc_auc']:.3f}")
    
    # Resilience score
    if 'resilience_score' in res:
        score = res['resilience_score']
        print(f"\n   Resilience Score: {score:.3f}")
        
        if score >= 0.90:
            print(f"      ✅ Excellent! Model is highly resilient")
        elif score >= 0.75:
            print(f"      🟡 Good, but shows some sensitivity to drift")
        elif score >= 0.60:
            print(f"      ⚠️  Moderate resilience - monitor closely")
        else:
            print(f"      ❌ Low resilience - consider retraining")
    
    # Drift detection
    if 'drift_detected' in res:
        drift = res['drift_detected']
        print(f"\n   Drift Detected: {'Yes ⚠️' if drift else 'No ✅'}")
    
    print("\n💡 Results stored in 'results' variable")
    print("   Use results.save_html() to generate reports!")
else:
    print("\n⚠️  Could not access detailed results")
    print("   But the test completed successfully!")

🔬 Running Resilience Test...

This will test your model against:
  • Data distribution shifts
  • Feature perturbations
  • Performance stability

Please wait...

✅ Resilience Tests Finished!
🎉 Test completed successfully: resilience

✅ RESILIENCE TEST COMPLETED!

💡 Generate an HTML report to see detailed visualizations!


## 4. 📊 Generate Interactive HTML Report

Create a comprehensive HTML report with:
- 📈 Interactive visualizations
- 📊 Detailed metrics and statistics
- 🔍 Feature-level analysis
- 💡 Actionable insights

In [7]:
print("📊 Generating Interactive HTML Report...\n")

# Create output directory
import os
output_dir = "outputs/resilience_reports"
os.makedirs(output_dir, exist_ok=True)

# Define report path
report_path = os.path.join(output_dir, "resilience_report_interactive.html")

# Generate interactive report using the results object
try:
    # Use the results object returned by run_tests()
    saved_path = results.save_html(
        test_type='resilience',
        file_path=report_path,
        model_name='Breast Cancer Classification',
        report_type='interactive'  # or 'static' for matplotlib charts
    )
    
    print("="*70)
    print("✅ REPORT GENERATED SUCCESSFULLY!")
    print("="*70)
    print(f"\n📁 Location: {saved_path}")
    print(f"\n💡 Open the report in your browser to explore:")
    print(f"   • Resilience scores and metrics")
    print(f"   • Model performance analysis")
    print(f"   • Feature distribution shifts")
    print(f"   • Drift detection results")
    print(f"   • Interactive charts and visualizations")
    print(f"\n🌐 To open: file://{os.path.abspath(saved_path)}")
    
except Exception as e:
    print("❌ Error generating report:")
    print(f"   {str(e)}")
    import traceback
    traceback.print_exc()

📊 Generating Interactive HTML Report...

❌ Error generating report:
   No test results available. Run tests first with experiment.run_tests()

💡 Make sure you ran the resilience test first!


### Optional: Generate Static Report

If you prefer static matplotlib charts instead of interactive Plotly visualizations:

In [8]:
# Optional: Generate static report with matplotlib
print("📊 Generating Static HTML Report...\n")

report_path_static = os.path.join(output_dir, "resilience_report_static.html")

try:
    # Use the results object returned by run_tests()
    saved_path = results.save_html(
        test_type='resilience',
        file_path=report_path_static,
        model_name='Breast Cancer Classification',
        report_type='static'
    )
    
    print("✅ Static report generated!")
    print(f"   📁 Location: {saved_path}")
    
except Exception as e:
    print(f"❌ Error: {str(e)}")
    import traceback
    traceback.print_exc()

📊 Generating Static HTML Report...

❌ Error: No test results available. Run tests first with experiment.run_tests()


---

## 🎯 Summary

In this notebook, you learned how to:

1. ✅ **Prepare your data** - Load dataset and train a model
2. ✅ **Create DBDataset** - Wrap your data and model
3. ✅ **Initialize Experiment** - Set up DeepBridge experiment
4. ✅ **Run resilience test** - Evaluate model robustness
5. ✅ **Generate reports** - Create interactive HTML visualizations

---

## 📚 Next Steps

Explore other DeepBridge tests:

- **Robustness Test** - How does your model handle noisy/perturbed features?
- **Uncertainty Test** - How confident is your model in its predictions?
- **Fairness Test** - Is your model fair across different groups?
- **Hyperparameter Test** - Which hyperparameters matter most?

```python
# Run multiple tests
results = exp.run_tests('quick')  # Runs all configured tests

# Or run specific tests
robustness_result = exp.run_test('robustness', config='quick')
uncertainty_result = exp.run_test('uncertainty', config='quick')
```

---

## 💡 Best Practices

1. **Always run resilience tests** before deploying to production
2. **Use 'quick' config** for rapid iteration during development
3. **Use 'full' config** for comprehensive analysis before deployment
4. **Monitor resilience scores** - scores < 0.7 indicate potential issues
5. **Review HTML reports** for detailed insights and actionable recommendations

---

**Remember: A resilient model is a reliable model in production!** 🚀